<a href="https://colab.research.google.com/github/wandb/examples/blob/master/colabs/mosaicml/MosaicML_Composer_and_wandb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<!--- @wandbcode{mosaicml} -->

<img src="https://wandb.me/logo-im-png" width="400" alt="Weights & Biases" />
<img src="https://raw.githubusercontent.com/mosaicml/composer/dev/docs/source/_static/images/header_dark.svg" width="400" alt="mosaicml" />

<!--- @wandbcode{mosaicml} -->

# Running fast with MosaicML Composer and Weight and Biases

[MosaicML Composer](https://docs.mosaicml.com) is a library for training neural networks better, faster, and cheaper. It contains many state-of-the-art methods for accelerating neural network training and improving generalization, along with an optional Trainer API that makes composing many different enhancements easy.

Coupled with [Weights & Biases integration](https://docs.wandb.ai/guides/integrations/composer), you can quickly train and monitor models for full traceability and reproducibility with only 2 extra lines of code:

```python
from composer import Trainer
from composer.loggers import WandBLogger

wandb_logger = WandBLogger(init_params=init_params)
trainer = Trainer(..., logger=wandb_logger)
```

W&B integration with Composer can automatically:
* log your configuration parameters
* log your losses and metrics
* log gradients and parameter distributions
* log your model
* keep track of your code
* log your system metrics (GPU, CPU, memory, temperature, etc)

### 🛠️ Installation and set-up

We need to install the following libraries:
* [mosaicml-composer](https://docs.mosaicml.com/en/v0.5.0/getting_started/installation.html) to set up and train our models
* [wandb](https://docs.wandb.ai/) to instrument our training

In [ ]:
!pip install -Uq wandb mosaicml

## Use the Composer `Trainer` class with Weights and Biases 🏋️‍♀️

W&B integration with MosaicML-Composer is built into the `Trainer` and can be configured to add extra functionalities through `WandBLogger`:

* logging of Artifacts: Use `log_artifacts=True` to log model checkpoints as `wandb.Artifacts`. You can setup how often by passing an int value to `log_artifacts_every_n_batches` (default = 100)
* you can also pass any parameter that you would pass to `wandb.init` in `init_params` as a dictionary. For example, you could pass `init_params = {"project":"try_mosaicml", "name":"benchmark", "entity":"user_name"}`.

For more details refer to [Logger documentation](https://docs.mosaicml.com/en/latest/api_reference/composer.loggers.wandb_logger.html#composer.loggers.wandb_logger.WandBLogger) and [Wandb docs](https://docs.wandb.ai)

In [ ]:
EPOCHS = 5
BS = 32

In [ ]:
import wandb

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

from composer import Callback, State, Logger, Trainer
from composer.models import mnist_model
from composer.loggers import WandBLogger
from composer.callbacks import SpeedMonitor, LRMonitor
from composer.algorithms import LabelSmoothing, CutMix, ChannelsLast

let's grab a copy of MNIST from `torchvision`

In [ ]:
transform = transforms.Compose([transforms.ToTensor()])
dataset = datasets.MNIST("data", train=True, download=True, transform=transform)
train_dataloader = DataLoader(dataset, batch_size=128)

we can import a simple ConvNet model to try

In [ ]:
model = mnist_model(num_classes=10)

### 📊 Tracking the experiment
> we define the `wandb.init` params here

In [ ]:
# config params to log
config = {"epochs":EPOCHS,
          "batch_size":BS,
          "model_name":"MNIST_Classifier"}

# these will get passed to wandb.init(**init_params)
wandb_init_kwargs = {"config":config}

# setup of the logger
wandb_logger = WandBLogger(project="mnist-composer",
                           log_artifacts=True,
                           init_kwargs=wandb_init_kwargs)

we are able to tweak what are we logging using `Callbacks` into the `Trainer` class.

In [ ]:
callbacks = [LRMonitor(),    # Logs the learning rate
             SpeedMonitor(), # Logs the training throughput
            ]

we include callbacks that measure the model throughput (and the learning rate) and logs them to Weights & Biases. [Callbacks](https://docs.mosaicml.com/en/latest/trainer/callbacks.html) control what is being logged, whereas loggers specify where the information is being saved. For more information on loggers, see [Logging](https://docs.mosaicml.com/en/latest/trainer/logging.html).

In [ ]:
trainer = Trainer(
    model=mnist_model(num_classes=10),
    train_dataloader=train_dataloader,
    max_duration="2ep",
    loggers=[wandb_logger],    # Pass your WandbLogger
    callbacks=callbacks,
    algorithms=[
        LabelSmoothing(smoothing=0.1),
        CutMix(alpha=1.0),
        ChannelsLast(),
        ]
)

once we are ready to train we call `fit`

In [ ]:
trainer.fit()

We close the Trainer to properly finish all callbacks and loggers

In [ ]:
trainer.close()

## ⚙️ Advanced: Using callbacks to log sample predictions

> Composer is extensible through its callback system.

We create a custom callback to automatically log sample predictions during validation.

In [ ]:
class LogPredictions(Callback):
    def __init__(self, num_samples=100):
        super().__init__()
        self.num_samples = num_samples
        self.data = []

    def batch_end(self, state: State, logger: Logger):
        """Compute predictions per batch and stores them on self.data"""
        if len(self.data) < self.num_samples:
            n = self.num_samples
            x, y = state.batch
            outputs = state.outputs.argmax(-1)
            data = [[wandb.Image(x_i), y_i, y_pred] for x_i, y_i, y_pred in list(zip(x[:n], y[:n], outputs[:n]))]
            self.data += data

    def epoch_end(self, state: State, logger: Logger):
        "Create a wandb.Table and logs it"
        columns = ['image', 'ground truth', 'prediction']
        table = wandb.Table(columns=columns, data=self.data[:self.num_samples])
        wandb.log({'predictions_table':table}, step=int(state.timestamp.batch))

we add `LogPredictions` to the other callbacks

In [ ]:
callbacks.append(LogPredictions())

In [ ]:
trainer.close()

In [ ]:
trainer = Trainer(
    model=mnist_model(num_classes=10),
    train_dataloader=train_dataloader,
    max_duration="2ep",
    loggers=[wandb_logger],    # Pass your WandbLogger
    callbacks=callbacks,
    algorithms=[
        LabelSmoothing(smoothing=0.1),
        CutMix(alpha=1.0),
        ChannelsLast(),
        ]
)

Once we're ready to train, we just call the `fit` method.

In [ ]:
trainer.fit()
trainer.close()

We can monitor losses, metrics, gradients, parameters and sample predictions as the model trains.

![composer.png](https://i.imgur.com/VFZLOB3.png?1)

## 📚 Resources

* We are excited to showcase this early support of [MosaicML-Composer](https://docs.mosaicml.com/en/latest/index.html) go ahead and try this new state of the art framework.

## ❓ Questions about W&B

If you have any questions about using W&B to track your model performance and predictions, please reach out to the [wandb community](https://community.wandb.ai).